In [ ]:
from open_ephys.analysis import Session
from matplotlib import pyplot as plt
import numpy as np

import consts
import importlib
importlib.reload(consts)

%matplotlib tk

In [ ]:

directory = '/home/pierfier/handata_server/eng_research_handata3/Yangyang_Wang/PV_V1_LED_SomArchon/109558_Vb_male/20240311/ephys/109558_flicker8hz_DBS40hz_fov1_2024-03-11_15-54-11/'

session = Session(directory)     



In [16]:
# Exploring
print(session.recordnodes[0].recordings[0])

data = session.recordnodes[0].recordings[0].continuous[0]

ephys_data = session.recordnodes[0].recordings[0].continuous[0]

# Extract and find the start times of recording trials
start_chan_idx = ephys_data.metadata['channel_names'].index('ADC5')
raw_start_trigger = ephys_data.samples[:, start_chan_idx]
frame_chan_idx = ephys_data.metadata['channel_names'].index('ADC1')
raw_frame_trigger = ephys_data.samples[:, frame_chan_idx]
stim_chan_idx = ephys_data.metadata['channel_names'].index('ADC4')
raw_stim_trigger = ephys_data.samples[:, stim_chan_idx]
flicker_chan_idx = ephys_data.metadata['channel_names'].index('ADC3')
raw_flicker_trigger = ephys_data.samples[:, flicker_chan_idx]
# Get all of the rise idxs
record_start_idx = consts.get_ephys_rise_indices(raw_start_trigger)
all_frame_idx = consts.get_ephys_rise_indices(raw_frame_trigger)
all_stim_idx = consts.get_ephys_rise_indices(raw_stim_trigger)
all_flicker_idx = consts.get_ephys_rise_indices(raw_flicker_trigger)
# Get the reverse flicker idxs
#TODO fix this and try to get the reverse indices for the flicker and get the pulse width
all_flicker_end_idx = consts.get_ephys_rise_indices(raw_flicker_trigger)
block_frame_timestamps = []
block_stim_timestamps = []
block_flicker_timestamps = []
#DEBUG
#plt.figure()
#plt.plot(ephys_data.timestamps, raw_start_trigger)
#plt.plot(ephys_data.timestamps[record_start_idx], raw_start_trigger[record_start_idx], '|')
#plt.title('Ephys Debugging')
#plt.show()
#plt.figure()
#plt.plot(ephys_data.timestamps, raw_start_trigger, '-g')
#plt.plot(ephys_data.timestamps, raw_stim_trigger, '-g')
#plt.plot(ephys_data.timestamps, raw_flicker_trigger, '-b')
# Loop through each recording start trigger and parse together all of the trials
for i in range(record_start_idx.shape[0]):
    start_idx = record_start_idx[i]
    if i == record_start_idx.shape[0] - 1:
        next_idx = float('inf')
    else:
        next_idx = record_start_idx[i + 1]
    # Find all of the camera frame indices for the given block
    frame_idxs = all_frame_idx[(all_frame_idx > start_idx) & (all_frame_idx < next_idx)]
    stim_idxs = all_stim_idx[(all_stim_idx > start_idx) & (all_stim_idx < next_idx)]
    flicker_idxs = all_flicker_idx[(all_flicker_idx > start_idx) & (all_flicker_idx < next_idx)]
    block_frame_timestamps.append(ephys_data.timestamps[frame_idxs])
    block_stim_timestamps.append(ephys_data.timestamps[stim_idxs])
    block_flicker_timestamps.append(ephys_data.timestamps[flicker_idxs])

# Check all of the stimulation and flicker start times
for frame_time, stim_time, flicker_time in zip(block_frame_timestamps, block_stim_timestamps, block_flicker_timestamps):
    tr_start = frame_time[0]
    frame_time = frame_time - tr_start
    stim_time = stim_time - tr_start
    flicker_time = flicker_time  - tr_start

    print(str(stim_time[0]) +' '+ str(flicker_time[0]))

Open Ephys GUI Recording
ID: 0x7f168099e150
Format: Binary
Directory: /home/pierfier/handata_server/eng_research_handata3/Yangyang_Wang/PV_V1_LED_SomArchon/109558_Vb_male/20240311/ephys/109558_flicker8hz_DBS40hz_fov1_2024-03-11_15-54-11/Record Node 103/experiment1/recording1
Experiment Index: 0
Recording Index: 0
1.998600000006718 0.9985666666902944
1.998600000132683 0.9985999998498301
1.9985999999698834 0.9985999999498745
1.998600000006718 0.9985666666902944
1.9986000001495086 0.9985666667416808
1.9986000000435524 0.9986000000235435
1.9985999999698834 0.9985999999498745
1.9985999998411899 0.9985666665247663
1.998600000006718 0.998599999986709
1.998599999970338 0.9985999998962143
1.998600000006718 0.9986000000235435


In [ ]:
# Print the data in the ADC 
all_data = data.get_samples(start_sample_index=0, end_sample_index=10000)
print(all_data.shape)

print(data.metadata['channel_names'])

# Get the rise times for camera frames
chan_ind = data.metadata['channel_names'].index('ADC1')
frame_idx = consts.get_ephys_rise_indices(data.samples[:, chan_ind])

frame_time = data.timestamps[frame_idx]

print('Unique frame difference')
print(np.unique(np.diff(frame_time)))

# Loop through and show all of the ADC channels data

plt.figure()

for i in range(1, 9):
    chan_ind = data.metadata['channel_names'].index('ADC' + str(i))
    adc = data.samples[:, chan_ind]
    min_d = np.min(adc)
    max_d = np.max(adc)
    norm_d = (adc - min_d) / (max_d - min_d)
    plt.plot(data.timestamps, norm_d + i*(1.25)) 

plt.show()

In [ ]:
plt.close('all')